# cjm-fasthtml-workflows

> Session-based workflow management for FastHTML applications with multi-step state tracking, job metadata storage, and operation deduplication.

## Install

```bash
pip install cjm_fasthtml_workflows
```

## Project Structure

```
nbs/
└── core/ (4)
    ├── deduplication.ipynb     # Track one-time operations per job to prevent duplicates.
    ├── helpers.ipynb           # Helper functions for common workflow patterns.
    ├── job_session.ipynb       # Manage job-related metadata that needs to be accessed across multiple requests.
    └── workflow_session.ipynb  # Namespaced workflow state management within FastHTML sessions.
```

Total: 4 notebooks across 1 directory

## Module Dependencies

```mermaid
graph LR
    core_deduplication[core.deduplication<br/>DeduplicationTracker]
    core_helpers[core.helpers<br/>Helpers]
    core_job_session[core.job_session<br/>JobSessionManager]
    core_workflow_session[core.workflow_session<br/>WorkflowSession]

    core_helpers --> core_deduplication
    core_helpers --> core_job_session
```

*2 cross-module dependencies detected*

## CLI Reference

No CLI commands found in this project.

## Module Overview

Detailed documentation for each module in the project:

### DeduplicationTracker (`deduplication.ipynb`)
> Track one-time operations per job to prevent duplicates.

#### Import

```python
from cjm_fasthtml_workflows.core.deduplication import (
    DeduplicationTracker
)
```
#### Classes

```python
class DeduplicationTracker:
    def __init__(
        self, 
        sess:Any, # FastHTML session object
        tracker_id:str # Unique identifier for this tracker (e.g., "saved_jobs", "sent_notifications")
    )
    "Track one-time operations per job to prevent duplicates."
    
    def __init__(
            self, 
            sess:Any, # FastHTML session object
            tracker_id:str # Unique identifier for this tracker (e.g., "saved_jobs", "sent_notifications")
        )
        "Initialize deduplication tracker."
    
    def is_processed(
            self, 
            job_id:str # Unique job identifier
        ) -> bool: # True if job has been marked as processed, False otherwise
        "Check if a job has been processed."
    
    def mark_processed(
            self, 
            job_id:str # Unique job identifier
        ) -> None
        "Mark a job as processed."
    
    def unmark_processed(
            self, 
            job_id:str # Unique job identifier
        ) -> None
        "Remove processed mark from a job (allows reprocessing)."
    
    def clear_all(self) -> None:
            """Clear all processed job IDs."""
            if self._key in self.sess
        "Clear all processed job IDs."
    
    def get_all_processed(self) -> Set[str]: # Set of job IDs that have been marked as processed
        "Get all processed job IDs."
```


### Helpers (`helpers.ipynb`)
> Helper functions for common workflow patterns.

#### Import

```python
from cjm_fasthtml_workflows.core.helpers import (
    get_job_session_info,
    once_per_job
)
```

#### Functions

```python
def get_job_session_info(
    job_id:str, # Unique job identifier
    job:Any, # Job object (should have attributes like file_name, file_path, plugin_id)
    sess:Any, # FastHTML session object
    fallback_fields:Optional[Dict[str, Any]]=None # Optional additional fallback fields
) -> Tuple[Dict[str, Any], Dict[str, Any]]: # (file_info, plugin_info) dictionaries
    "Retrieve job-related display information from session with fallbacks."
```

```python
def once_per_job(
    sess:Any, # FastHTML session object
    job_id:str, # Unique job identifier
    operation:Callable[[], Any], # Callable to execute (should take no arguments)
    tracker_id:str="default_operation" # Unique identifier for this operation type
) -> Optional[Any]: # Result of operation if executed, None if already processed
    "Execute an operation once per job using automatic deduplication."
```


### JobSessionManager (`job_session.ipynb`)
> Manage job-related metadata that needs to be accessed across multiple requests.

#### Import

```python
from cjm_fasthtml_workflows.core.job_session import (
    JobSessionManager
)
```
#### Classes

```python
class JobSessionManager:
    def __init__(
        self, 
        sess:Any, # FastHTML session object
        prefix:str="job" # Prefix for job keys in session
    )
    "Manage job-related metadata that needs to be accessed across multiple requests."
    
    def __init__(
            self, 
            sess:Any, # FastHTML session object
            prefix:str="job" # Prefix for job keys in session
        )
        "Initialize job session manager."
    
    def store_job_metadata(
            self, 
            job_id:str, # Unique job identifier
            metadata:Dict[str, Any] # Dictionary of metadata to store
        ) -> None
        "Store metadata for a job."
    
    def get_job_metadata(
            self, 
            job_id:str, # Unique job identifier
            default:Optional[Dict[str, Any]]=None # Default value if metadata not found
        ) -> Dict[str, Any]: # Job metadata dictionary
        "Retrieve metadata for a job."
    
    def delete_job_metadata(
            self, 
            job_id:str # Unique job identifier
        ) -> None
        "Remove metadata for a job."
    
    def clear_all_jobs(self) -> None:
            """Clear all job metadata from session."""
            keys_to_delete = [k for k in self.sess.keys() if k.startswith(f"{self.prefix}_")]
            for key in keys_to_delete
        "Clear all job metadata from session."
```


### WorkflowSession (`workflow_session.ipynb`)
> Namespaced workflow state management within FastHTML sessions.

#### Import

```python
from cjm_fasthtml_workflows.core.workflow_session import (
    WorkflowSession
)
```
#### Classes

```python
class WorkflowSession:
    def __init__(
        self, 
        sess:Any, # FastHTML session object (from route handler parameter)
        workflow_id:str # Unique identifier for this workflow (e.g., "registration", "checkout")
    )
    "Manage workflow state within FastHTML sessions with proper namespacing."
    
    def __init__(
            self, 
            sess:Any, # FastHTML session object (from route handler parameter)
            workflow_id:str # Unique identifier for this workflow (e.g., "registration", "checkout")
        )
        "Initialize workflow session wrapper."
    
    def set(
            self, 
            key:str, # Key name within this workflow namespace
            value:Any # Value to store (must be JSON-serializable for session storage)
        ) -> None
        "Store value in workflow session."
    
    def get(
            self, 
            key:str, # Key name within this workflow namespace
            default:Any=None # Default value if key not found
        ) -> Any: # Stored value or default
        "Retrieve value from workflow session."
    
    def delete(
            self, 
            key:str # Key name within this workflow namespace
        ) -> None
        "Remove value from workflow session."
    
    def clear(self) -> None:
            """Clear all data for this workflow from session."""
            keys_to_delete = [k for k in self.sess.keys() if k.startswith(self._prefix)]
            for key in keys_to_delete
        "Clear all data for this workflow from session."
    
    def get_all(self) -> Dict[str, Any]: # Dictionary mapping original keys to their values
            """Get all workflow data as a dictionary with original (unprefixed) keys."""
            result = {}
            prefix_len = len(self._prefix)
            for key, value in self.sess.items()
        "Get all workflow data as a dictionary with original (unprefixed) keys."
    
    def has(
            self, 
            key:str # Key name within this workflow namespace
        ) -> bool: # True if key exists, False otherwise
        "Check if key exists in workflow session."
```
